# Imports

In [1]:
!pip install fire easyocr pillow transformers==4.19.2 keras-ocr kaggle pytorch_lightning
!pip install python-magic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 48.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 76.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.9/776.9 kB 59.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 92.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 69.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 35.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.1/806.1 kB 71.2 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.5.0-py2.py3-none-any.whl size

In [ ]:
!pip install git+https://github.com/openai/CLIP.git
!pip install git+https://github.com/openai/whisper.git

!wget https://raw.githubusercontent.com/oyyd/frozen_east_text_detection.pb/master/frozen_east_text_detection.pb
!wget https://github.com/miccunifi/ISSUES/releases/download/latest/hmc_text-inv-comb_best.ckpt
!wget https://github.com/miccunifi/ISSUES/releases/download/latest/resources.zip
!unzip resources.zip

!gdown 1JhT8L7F-RnRQJTfifOndi_-TsukM-ErC # Download Image Model
!gdown 1IdaBtMSvtyzF0ByVaBHtvM0JYSXRExRX # Download COCO Weights
!mv '/content/hmc_text-inv-comb_best.ckpt' '/content/resources/pretrained_models'

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-97t3cy93
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-97t3cy93
  Resolved https://github.com/openai/CLIP.git to commit a1d071733d7111c9c014f024669f959182114e33
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 1.5 MB/s eta 0:00:00
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369497 sha256=ee66124c3d52fe0fc0c76bcb4e77d96477d865b0d7724f05b5b0e82e7dfb4a78
  Stored in directory: /tmp/pip-ephem-wheel-cache-svtjsy9t/wheels/da/2b/4c/d6691fa9597aac8bb85d2ac13b112deb897d5b50f5ad9a37e4
Successfully built clip
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-3oz2ilp8
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-3oz2ilp8
  Resolved https://github.com/openai/whisper.git to commit e58f28804528831904c3b6f2c0e473f34

In [9]:
import numpy as np
import time
import copy
import os
import shutil
import torch.nn as nn
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import torch
import whisper
import clip
import torch.nn.functional as nnf
import sys
import IPython.display as display
import skimage.io as io
import random
import pickle as pkl
import json
import easyocr
import fire
import cv2
import glob
import math
import keras_ocr
import argparse
import pytorch_lightning as pl
import torchmetrics

from transformers import ResNetForImageClassification
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import load_model
from tqdm.auto import tqdm
from torch.utils.data import DataLoader
from torchvision.utils import save_image
from transformers import CLIPTokenizer, CLIPProcessor
from typing import Tuple, List, Union, Optional
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm, trange
from google.colab import files, drive
from IPython.display import Image
from PIL import Image, ImageDraw
from skimage import transform
from skimage.feature import canny
from skimage.color import rgb2gray, gray2rgb
from multiprocessing import Pool
from distutils.archive_util import make_archive
from imutils.object_detection import non_max_suppression
from clip.model import CLIP
from torchvision import transforms

# Mount Google Drive

In [ ]:
drive.mount('/content/drive')

# Load Text Model

In [ ]:
text_model = AutoModelForSequenceClassification.from_pretrained("Kidsshield/CBD-FullData")
tokenizer = AutoTokenizer.from_pretrained('Kidsshield/CBD-FullData')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
text_model.to(device)

# Load MEME Text Model

In [ ]:
meme_text_model = AutoModelForSequenceClassification.from_pretrained("Kidsshield/MEME")
tokenizer = AutoTokenizer.from_pretrained('Kidsshield/MEME')

meme_text_model.to(device)

# Load Image Model

In [ ]:
image_model = load_model('/content/seperatedclassesmodel.h5')

# Text + Image Inference

In [ ]:
def perform_inference(input, meme=False):
    if os.path.isfile(input):
        return image_inference(input)
    else:
        if meme:
            return text_inference(input)
        else:
            return text_inference(input)


def text_inference(input):
    encoding = tokenizer(input, return_tensors='pt')
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    with torch.no_grad():
        output = text_model(input_ids=input_ids, attention_mask=attention_mask)
    prediction = 'Bully' if torch.argmax(output.logits).item() == 1 else 'NonBully'
    print("Prediction:", prediction)
    return prediction


def meme_text_inference(input):
    encoding = tokenizer(input, return_tensors='pt')
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    with torch.no_grad():
        output = meme_text_model(input_ids=input_ids, attention_mask=attention_mask)
    prediction = 'Bully' if torch.argmax(output.logits).item() == 1 else 'NonBully'
    print("Prediction:", prediction)
    return prediction


def image_inference(input):

    dic = {'drawing': 0, 'hentai': 1,'neutral' : 2,'porn':3,'sexy':4,'violence':5,'weapon':6}
    icd = {v: k for k, v in dic.items()}
    img = load_img(input, target_size=(224, 224))
    img = img_to_array(img)
    img = img / 255.0
    img = np.expand_dims(img, axis=0)
    predictions = image_model.predict(img)
    predicted_class = np.argmax(predictions, axis=1)
    predicted_label = icd[predicted_class[0]]
    probability = round(np.max(predictions) * 100, 2)
    print("Prediction:", predicted_label)

#Load Audio model

In [ ]:
audio_model = whisper.load_model("base")

#MEME Classification

In [ ]:
# #set GPU id
# CUDA_DEVICE=0
# torch.cuda.set_device(CUDA_DEVICE)
# device = torch.device("cuda:"+str(CUDA_DEVICE))

In [ ]:
model_path = os.path.join('/content/coco_weights.pt')

In [ ]:
N = type(None)
V = np.array
ARRAY = np.ndarray
ARRAYS = Union[Tuple[ARRAY, ...], List[ARRAY]]
VS = Union[Tuple[V, ...], List[V]]
VN = Union[V, N]
VNS = Union[VS, N]
T = torch.Tensor
TS = Union[Tuple[T, ...], List[T]]
TN = Optional[T]
TNS = Union[Tuple[TN, ...], List[TN]]
TSN = Optional[TS]
TA = Union[T, ARRAY]
D = torch.device

In [ ]:
#@title Model

class MLP(nn.Module):

    def forward(self, x: T) -> T:
        return self.model(x)

    def __init__(self, sizes: Tuple[int, ...], bias=True, act=nn.Tanh):
        super(MLP, self).__init__()
        layers = []
        for i in range(len(sizes) -1):
            layers.append(nn.Linear(sizes[i], sizes[i + 1], bias=bias))
            if i < len(sizes) - 2:
                layers.append(act())
        self.model = nn.Sequential(*layers)


class ClipCaptionModel(nn.Module):

    #@functools.lru_cache #FIXME
    def get_dummy_token(self, batch_size: int, device: D) -> T:
        return torch.zeros(batch_size, self.prefix_length, dtype=torch.int64, device=device)

    def forward(self, tokens: T, prefix: T, mask: Optional[T] = None, labels: Optional[T] = None):
        embedding_text = self.gpt.transformer.wte(tokens)
        prefix_projections = self.clip_project(prefix).view(-1, self.prefix_length, self.gpt_embedding_size)
        #print(embedding_text.size()) #torch.Size([5, 67, 768])
        #print(prefix_projections.size()) #torch.Size([5, 1, 768])
        embedding_cat = torch.cat((prefix_projections, embedding_text), dim=1)
        if labels is not None:
            dummy_token = self.get_dummy_token(tokens.shape[0], tokens.device)
            labels = torch.cat((dummy_token, tokens), dim=1)
        out = self.gpt(inputs_embeds=embedding_cat, labels=labels, attention_mask=mask)
        return out

    def __init__(self, prefix_length: int, prefix_size: int = 512):
        super(ClipCaptionModel, self).__init__()
        self.prefix_length = prefix_length
        self.gpt = GPT2LMHeadModel.from_pretrained('gpt2')
        self.gpt_embedding_size = self.gpt.transformer.wte.weight.shape[1]
        if prefix_length > 10:  # not enough memory
            self.clip_project = nn.Linear(prefix_size, self.gpt_embedding_size * prefix_length)
        else:
            self.clip_project = MLP((prefix_size, (self.gpt_embedding_size * prefix_length) // 2, self.gpt_embedding_size * prefix_length))


class ClipCaptionPrefix(ClipCaptionModel):

    def parameters(self, recurse: bool = True):
        return self.clip_project.parameters()

    def train(self, mode: bool = True):
        super(ClipCaptionPrefix, self).train(mode)
        self.gpt.eval()
        return self

In [ ]:
#@title Caption prediction

def generate_beam(model, tokenizer, beam_size: int = 5, prompt=None, embed=None,
                  entry_length=67, temperature=1., stop_token: str = '.'):

    model.eval()
    stop_token_index = tokenizer.encode(stop_token)[0]
    tokens = None
    scores = None
    device = next(model.parameters()).device
    seq_lengths = torch.ones(beam_size, device=device)
    is_stopped = torch.zeros(beam_size, device=device, dtype=torch.bool)
    with torch.no_grad():
        if embed is not None:
            generated = embed
        else:
            if tokens is None:
                tokens = torch.tensor(tokenizer.encode(prompt))
                tokens = tokens.unsqueeze(0).to(device)
                generated = model.gpt.transformer.wte(tokens)
        for i in range(entry_length):
            outputs = model.gpt(inputs_embeds=generated)
            logits = outputs.logits
            logits = logits[:, -1, :] / (temperature if temperature > 0 else 1.0)
            logits = logits.softmax(-1).log()
            if scores is None:
                scores, next_tokens = logits.topk(beam_size, -1)
                generated = generated.expand(beam_size, *generated.shape[1:])
                next_tokens, scores = next_tokens.permute(1, 0), scores.squeeze(0)
                if tokens is None:
                    tokens = next_tokens
                else:
                    tokens = tokens.expand(beam_size, *tokens.shape[1:])
                    tokens = torch.cat((tokens, next_tokens), dim=1)
            else:
                logits[is_stopped] = -float(np.inf)
                logits[is_stopped, 0] = 0
                scores_sum = scores[:, None] + logits
                seq_lengths[~is_stopped] += 1
                scores_sum_average = scores_sum / seq_lengths[:, None]
                scores_sum_average, next_tokens = scores_sum_average.view(-1).topk(beam_size, -1)
                next_tokens_source = next_tokens // scores_sum.shape[1]
                seq_lengths = seq_lengths[next_tokens_source]
                next_tokens = next_tokens % scores_sum.shape[1]
                next_tokens = next_tokens.unsqueeze(1)
                tokens = tokens[next_tokens_source]
                tokens = torch.cat((tokens, next_tokens), dim=1)
                generated = generated[next_tokens_source]
                scores = scores_sum_average * seq_lengths
                is_stopped = is_stopped[next_tokens_source]
            next_token_embed = model.gpt.transformer.wte(next_tokens.squeeze()).view(generated.shape[0], 1, -1)
            generated = torch.cat((generated, next_token_embed), dim=1)
            is_stopped = is_stopped + next_tokens.eq(stop_token_index).squeeze()
            if is_stopped.all():
                break
    scores = scores / seq_lengths
    output_list = tokens.cpu().numpy()
    output_texts = [tokenizer.decode(output[:int(length)]) for output, length in zip(output_list, seq_lengths)]
    order = scores.argsort(descending=True)
    output_texts = [output_texts[i] for i in order]
    return output_texts


def generate2(
        model,
        tokenizer,
        tokens=None,
        prompt=None,
        embed=None,
        entry_count=1,
        entry_length=67,  # maximum number of words
        top_p=0.8,
        temperature=1.,
        stop_token: str = '.',
):
    model.eval()
    generated_num = 0
    generated_list = []
    stop_token_index = tokenizer.encode(stop_token)[0]
    filter_value = -float("Inf")
    device = next(model.parameters()).device

    with torch.no_grad():

        for entry_idx in trange(entry_count):
            if embed is not None:
                generated = embed
            else:
                if tokens is None:
                    tokens = torch.tensor(tokenizer.encode(prompt))
                    tokens = tokens.unsqueeze(0).to(device)

                generated = model.gpt.transformer.wte(tokens)

            for i in range(entry_length):

                outputs = model.gpt(inputs_embeds=generated)
                logits = outputs.logits
                logits = logits[:, -1, :] / (temperature if temperature > 0 else 1.0)
                sorted_logits, sorted_indices = torch.sort(logits, descending=True)
                cumulative_probs = torch.cumsum(nnf.softmax(sorted_logits, dim=-1), dim=-1)
                sorted_indices_to_remove = cumulative_probs > top_p
                sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[
                                                    ..., :-1
                                                    ].clone()
                sorted_indices_to_remove[..., 0] = 0

                indices_to_remove = sorted_indices[sorted_indices_to_remove]
                logits[:, indices_to_remove] = filter_value
                next_token = torch.argmax(logits, -1).unsqueeze(0)
                next_token_embed = model.gpt.transformer.wte(next_token)
                if tokens is None:
                    tokens = next_token
                else:
                    tokens = torch.cat((tokens, next_token), dim=1)
                generated = torch.cat((generated, next_token_embed), dim=1)
                if stop_token_index == next_token.item():
                    break

            output_list = list(tokens.squeeze().cpu().numpy())
            output_text = tokenizer.decode(output_list)
            generated_list.append(output_text)

    return generated_list[0]

In [ ]:
clip_model, preprocess = clip.load("ViT-B/32", device='cuda', jit=False)
clip_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

In [ ]:
#@title Load model weights
prefix_length = 10
meme_model = ClipCaptionModel(prefix_length)
meme_model.load_state_dict(torch.load(model_path, map_location=device))
meme_model = meme_model.eval()
meme_model = meme_model.to(device)

In [ ]:
#@title Inpainting image
"""
src: https://pyimagesearch.com/2018/08/20/opencv-text-detection-east-text-detector/
"""
source_dir = '/content/'
target_dir_masked = '/content/'
target_dir_inpainted = '/content/'
east_path = '/content/frozen_east_text_detection.pb'
width = 320
height = 320
min_confidence = 0.5
# img_fns = [x for x in os.listdir(source_dir) if x.endswith('.jpg') or x.endswith('.png') or x.endswith('.jpeg')]

def transform_image(img_fp):

    image = cv2.imread(img_fp)
    masked, inpainted = image.copy(), image.copy()
    (H, W) = image.shape[:2]
    (newW, newH) = (width, height)
    rW = W / float(newW)
    rH = H / float(newH)
    image = cv2.resize(image, (newW, newH))
    (H, W) = image.shape[:2]

    blob = cv2.dnn.blobFromImage(image, 1.0, (W, H),
        (123.68, 116.78, 103.94), swapRB=True, crop=False)
    net.setInput(blob)
    (scores, geometry) = net.forward(layerNames)

    (numRows, numCols) = scores.shape[2:4]
    rects = []
    confidences = []

    for y in range(0, numRows):

        scoresData = scores[0, 0, y]
        xData0 = geometry[0, 0, y]
        xData1 = geometry[0, 1, y]
        xData2 = geometry[0, 2, y]
        xData3 = geometry[0, 3, y]
        anglesData = geometry[0, 4, y]


        for x in range(0, numCols):

            if scoresData[x] < min_confidence:
                continue

            (offsetX, offsetY) = (x * 4.0, y * 4.0)
            angle = anglesData[x]
            cos = np.cos(angle)
            sin = np.sin(angle)
            h = xData0[x] + xData2[x]
            w = xData1[x] + xData3[x]
            endX = int(offsetX + (cos * xData1[x]) + (sin * xData2[x]))
            endY = int(offsetY - (sin * xData1[x]) + (cos * xData2[x]))
            startX = int(endX - w)
            startY = int(endY - h)
            rects.append((startX, startY, endX, endY))
            confidences.append(scoresData[x])

    boxes = non_max_suppression(np.array(rects), probs=confidences)
    mask_for_inpainting = np.zeros(inpainted.shape[:2], np.uint8)
    for (startX, startY, endX, endY) in boxes:

        startX = int(startX * rW)
        startY = int(startY * rH)
        endX = int(endX * rW)
        endY = int(endY * rH)
        cv2.rectangle(masked, (startX, startY), (endX, endY), (127, 127, 127), -1)
        cv2.rectangle(mask_for_inpainting, (startX, startY), (endX, endY), 255, -1)

    inpainted = cv2.inpaint(inpainted, mask_for_inpainting, 7, cv2.INPAINT_NS)

    return masked, inpainted


layerNames = ["feature_fusion/Conv_7/Sigmoid", "feature_fusion/concat_3"]
print("[INFO] loading EAST text detector...")
net = cv2.dnn.readNet(east_path)

def transform_and_save_image(img_fp):
    path, img_fn = os.path.split(img_fp)
    img_fp_inpainted = os.path.join(path, "inpaint_"+img_fn)

    img_masked, img_inpainted = transform_image(img_fp)
    return img_inpainted

In [ ]:
#@title Inference
use_beam_search = True
reader = easyocr.Reader(['en'])

def process_meme_caption(file_path):
    head, img_fn = os.path.split(file_path)
    image = transform_and_save_image(file_path)
    pil_image = PIL.Image.fromarray(image)


    image = preprocess(pil_image).unsqueeze(0).to(device)
    with torch.no_grad():
        prefix = clip_model.encode_image(image).to(device, dtype=torch.float32)
        prefix_embed = meme_model.clip_project(prefix).reshape(1, prefix_length, -1)
    if use_beam_search:
        generated_text_prefix = generate_beam(meme_model, clip_tokenizer, embed=prefix_embed)[0]
    else:
        generated_text_prefix = generate2(meme_model, clip_tokenizer, embed=prefix_embed)

    return generated_text_prefix

# ISSUES MEME Inference

Keras - OCR Module

In [ ]:
# https://shegocodes.medium.com/extract-text-from-image-left-to-right-and-top-to-bottom-with-keras-ocr-b56f098a6efe

In [ ]:
def get_distance(predictions):
    """
    Function returns dictionary with (key,value):
        * text : detected text in image
        * center_x : center of bounding box (x)
        * center_y : center of bounding box (y)
        * distance_from_origin : hypotenuse
        * distance_y : distance between y and origin (0,0)
    """

    x0, y0 = 0, 0

    detections = []
    for group in predictions:

        top_left_x, top_left_y = group[1][0]
        bottom_right_x, bottom_right_y = group[1][1]
        center_x, center_y = (top_left_x + bottom_right_x)/2, (top_left_y + bottom_right_y)/2

        distance_from_origin = math.dist([x0,y0], [center_x, center_y])

        distance_y = center_y - y0

        detections.append({
                            'text': group[0],
                            'center_x': center_x,
                            'center_y': center_y,
                            'distance_from_origin': distance_from_origin,
                            'distance_y': distance_y
                        })

    return detections


def distinguish_rows(lst, thresh=15):
    """Function to help distinguish unique rows"""
    sublists = []
    for i in range(0, len(lst)-1):
        if (lst[i+1]['distance_y'] - lst[i]['distance_y'] <= thresh):
            if lst[i] not in sublists:
                sublists.append(lst[i])
            sublists.append(lst[i+1])
        else:
            yield sublists
            sublists = [lst[i+1]]
    yield sublists

In [ ]:
pipeline = keras_ocr.pipeline.Pipeline()

In [ ]:
def MyOCR(image_path):

  read_image = keras_ocr.tools.read(image_path)
  prediction_groups = pipeline.recognize([read_image])
  predictions = prediction_groups[0]
  predictions = get_distance(predictions)
  predictions = list(distinguish_rows(predictions, thresh=15))
  ordered_preds = []
  for row in predictions:
    row = sorted(row, key=lambda x:x['distance_from_origin'])
    for each in row:
      ordered_preds.append(each['text'])
    last = ' '.join(ordered_preds)


  return last

ISSUES INFERENCE

In [ ]:
class Combiner(nn.Module):
    def __init__(self, convex_tensor: bool, input_dim: int, comb_proj: bool, comb_fusion: str):
        super(Combiner, self).__init__()
        self.map_dim = input_dim
        self.comb_proj = comb_proj
        self.comb_fusion = comb_fusion
        self.convex_tensor = convex_tensor

        if self.convex_tensor:
            branch_out_dim = self.map_dim
        else:
            branch_out_dim = 1

        comb_in_dim = self.map_dim
        comb_concat_out_dim = comb_in_dim

        if self.comb_proj:
            self.comb_image_proj = nn.Sequential(
                nn.Linear(comb_in_dim, 2 * comb_in_dim),
                nn.ReLU(),
                nn.Dropout(p=0.5)
            )

            self.comb_text_proj = nn.Sequential(
                nn.Linear(comb_in_dim, 2 * comb_in_dim),
                nn.ReLU(),
                nn.Dropout(p=0.5)
            )

            comb_in_dim = 2 * comb_in_dim

        if self.comb_fusion == 'concat':
            branch_in_dim = 2 * comb_in_dim
        elif self.comb_fusion == 'align':
            branch_in_dim = comb_in_dim
        else:
            ValueError()

        self.comb_shared_branch = nn.Sequential(
            nn.Linear(branch_in_dim, 2 * branch_in_dim),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(2 * branch_in_dim, branch_out_dim),
            nn.Sigmoid()
        )

        self.comb_concat_branch = nn.Sequential(
            nn.Linear(branch_in_dim, 2 * branch_in_dim),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(2 * branch_in_dim, comb_concat_out_dim),
        )

    def __call__(self, *args, **kwargs):
        return super().__call__(*args, **kwargs)

    def forward(self, img_projection, post_features):
        if self.comb_proj:
            proj_img_fea = self.comb_image_proj(img_projection)
            proj_txt_fea = self.comb_text_proj(post_features)
        else:
            proj_img_fea = img_projection
            proj_txt_fea = post_features

        if self.comb_fusion == 'concat':
            comb_features = torch.cat([proj_img_fea, proj_txt_fea], dim=1)
        elif self.comb_fusion == 'align':
            comb_features = torch.mul(proj_img_fea, proj_txt_fea)
        else:
            raise ValueError()

        side_branch = self.comb_shared_branch(comb_features)
        central_branch = self.comb_concat_branch(comb_features)

        features = central_branch + ((1 - side_branch) * img_projection + side_branch * post_features)

        return features



PHI_INPUT_DIM = 768


class TextualInversion(nn.Module):
    def __init__(self, clip_model: CLIP, clip_img_enc_output_dim: int, phi_proj: bool, text_proj: bool, post_proj: bool,
                 drop_probs, phi_freeze: bool, enh_text: bool, post_dim=None, num_pre_proj_layers=1):
        super(TextualInversion, self).__init__()

        self.clip_model = clip_model
        self.phi_proj = phi_proj
        self.text_proj = text_proj
        self.post_proj = post_proj
        self.enh_text = enh_text

        assert self.clip_model.token_embedding.embedding_dim == PHI_INPUT_DIM, 'CLIP model selected is not compatible' \
                                                                               ' with the pre-trained phi network'

        if self.post_proj:
            self.output_dim = post_dim
        else:
            self.output_dim = self.clip_model.token_embedding.embedding_dim

        phi_layers = [nn.Linear(PHI_INPUT_DIM, 3072),
                      nn.GELU(),
                      nn.Dropout(p=0.5),
                      nn.Linear(3072, 3072),
                      nn.GELU(),
                      nn.Dropout(p=0.5),
                      nn.Linear(3072, PHI_INPUT_DIM),
                      ]
        self.phi = nn.Sequential(*phi_layers)

        if phi_proj:

            phi_map_layers = [nn.Linear(PHI_INPUT_DIM, PHI_INPUT_DIM),
                              nn.Dropout(p=drop_probs[0])]
            self.phi_map = nn.Sequential(*phi_map_layers)

        phi_dict = torch.load("./resources/pretrained_weights/phi/phi_imagenet_45.pt")["MLPCustom"]
        with torch.no_grad():
            self.phi[0].weight.copy_(phi_dict['layers.0.weight'])
            self.phi[0].bias.copy_(phi_dict['layers.0.bias'])
            self.phi[3].weight.copy_(phi_dict['layers.3.weight'])
            self.phi[3].bias.copy_(phi_dict['layers.3.bias'])
            self.phi[6].weight.copy_(phi_dict['layers.6.weight'])
            self.phi[6].bias.copy_(phi_dict['layers.6.bias'])

        if phi_freeze:
            for name, p in self.phi.named_parameters():
                p.requires_grad_(False)

        in_dim = clip_img_enc_output_dim
        pre_inversion_layers = [nn.Linear(in_dim, PHI_INPUT_DIM),
                                nn.Dropout(p=drop_probs[0])]
        for _ in range(1, num_pre_proj_layers):
            pre_inversion_layers.extend(
                [nn.ReLU(), nn.Linear(PHI_INPUT_DIM, PHI_INPUT_DIM), nn.Dropout(p=drop_probs[0])])
        self.pre_inversion_map = nn.Sequential(*pre_inversion_layers)

        if post_proj:

            post_inversion_layers = [nn.Linear(self.clip_model.token_embedding.embedding_dim, post_dim),
                                     nn.Dropout(p=drop_probs[0])]
            self.post_inversion_map = nn.Sequential(*post_inversion_layers)

    def encode_with_vstar(self, clip_model: CLIP, text: torch.tensor, v_star: torch.tensor, num_vstar=1,
                          pooling=True, token_id=259, proj=True):
        x = clip_model.token_embedding(text).type(clip_model.dtype)
        _, counts = torch.unique((text == token_id).nonzero(as_tuple=True)[0], return_counts=True)
        cum_sum = torch.cat((torch.zeros(1, device=torch.device('cuda')).int(), torch.cumsum(counts, dim=0)[:-1]))
        first_vstar_indexes = (text == token_id).nonzero()[cum_sum][:, 1]
        rep_idx = torch.cat([(first_vstar_indexes + n).unsqueeze(0) for n in range(num_vstar)])

        if v_star.shape[0] == x.shape[0]:
            if len(v_star.shape) == 2:
                v_star = v_star.unsqueeze(1)
            x[torch.arange(x.shape[0]).repeat_interleave(num_vstar).reshape(x.shape[0],
                                                                            num_vstar), rep_idx.T] = v_star.to(
                x.dtype)
        else:
            raise ValueError()

        x = x + clip_model.positional_embedding.type(clip_model.dtype)
        x = x.permute(1, 0, 2)
        x = clip_model.transformer(x)
        x = x.permute(1, 0, 2)
        x = clip_model.ln_final(x).type(clip_model.dtype)

        if pooling:
            if proj:
                x = x[torch.arange(x.shape[0]), text.argmax(dim=-1)] @ clip_model.text_projection
            else:
                x = x[torch.arange(x.shape[0]), text.argmax(dim=-1)]
        return x

    def __call__(self, *args, **kwargs):
        return super().__call__(*args, **kwargs)

    def forward(self, prompt, image_features):
        img_features = self.pre_inversion_map(image_features)

        v_star = self.phi(img_features)

        if self.phi_proj:
            v_star = self.phi_map(v_star)

        text_input = prompt

        features = self.encode_with_vstar(self.clip_model, text_input, v_star, proj=self.text_proj).float()

        if self.post_proj:
            features = self.post_inversion_map(features)


        return features

CLIP_IMG_ENC_OUTPUT_DIM_BEFORE_PROJ = 1024


class LinearProjection(nn.Module):
    def __init__(self, input_dim, output_dim, num_layers, drop_probs):
        super(LinearProjection, self).__init__()

        map_layers = [nn.Linear(input_dim, output_dim),
                      nn.Dropout(p=drop_probs[0])]

        for _ in range(1, num_layers):
            map_layers.extend(
                [nn.ReLU(), nn.Linear(output_dim, output_dim), nn.Dropout(p=drop_probs[0])])

        self.proj = nn.Sequential(*map_layers)

    def __call__(self, *args, **kwargs):
        return super().__call__(*args, **kwargs)

    def forward(self, x):
        return self.proj(x)


class HateClassifier(pl.LightningModule):

    def __init__(self, args):
        super().__init__()

        self.dataset = args.dataset
        self.num_mapping_layers = args.num_mapping_layers
        self.map_dim = args.map_dim
        self.fusion = args.fusion
        self.num_pre_output_layers = args.num_pre_output_layers
        self.lr = args.lr
        self.weight_decay = args.weight_decay
        self.batch_size = args.batch_size

        self.name = args.name
        self.fast_process = args.fast_process

        self.proj_map = args.proj_map

        self.pretrained_proj = args.pretrained_proj_weights
        self.freeze_proj = args.freeze_proj_layers

        self.convex_tensor = args.convex_tensor
        self.comb_proj = args.comb_proj
        self.comb_fusion = args.comb_fusion

        self.enh_text = args.enh_text
        self.phi_freeze = args.phi_freeze
        self.text_inv_proj = args.text_inv_proj
        self.phi_inv_proj = args.phi_inv_proj
        self.post_inv_proj = args.post_inv_proj

        self.acc = torchmetrics.Accuracy(task='binary')
        self.auroc = torchmetrics.AUROC(task='binary')

        self.pretrained_weights_path = f'./resources/pretrained_weights/{self.dataset}'

        self.clip_model, _ = clip.load("ViT-L/14", device="cuda", jit=False)

        self.clip_model.visual.proj = None
        self.clip_model.float()

        for _, p in self.clip_model.named_parameters():
            p.requires_grad_(False)

        self.image_map = LinearProjection(CLIP_IMG_ENC_OUTPUT_DIM_BEFORE_PROJ, self.map_dim,
                                          self.num_mapping_layers, args.drop_probs)
        self.text_map = LinearProjection(self.clip_model.token_embedding.embedding_dim, self.map_dim,
                                         self.num_mapping_layers, args.drop_probs)

        if self.name in ['hate-clipper', 'adaptation']:
            if args.fusion == 'align':
                pre_output_input_dim = self.map_dim
            elif args.fusion == 'concat':
                pre_output_input_dim = self.map_dim * 2
        elif self.name == 'text-only':
            if self.proj_map:
                pre_output_input_dim = self.map_dim
            else:
                pre_output_input_dim = self.clip_model.token_embedding.embedding_dim
        elif self.name == 'image-only':
            if self.proj_map:
                pre_output_input_dim = self.map_dim
            else:
                pre_output_input_dim = CLIP_IMG_ENC_OUTPUT_DIM_BEFORE_PROJ

        elif self.name == 'sum':
            # proj_map is used by default
            pre_output_input_dim = self.map_dim

        elif self.name == 'combiner':
            # proj_map is used by default
            self.comb = Combiner(self.convex_tensor, self.map_dim, self.comb_proj, self.comb_fusion)

            if self.pretrained_proj:
                # Load pre-trained weights
                assert self.num_mapping_layers == 1
                if self.dataset == 'hmc':
                    assert self.map_dim in [1024, 768]
                    weights = f'hmc_{self.map_dim}_projection_embeddings.pt'
                elif self.dataset == 'harmeme':
                    assert self.map_dim == 768
                    weights = f'harmeme_{self.map_dim}_projection_embeddings.pt'
                else:
                    raise ValueError()

                state_dict = torch.load(f'{self.pretrained_weights_path}/{weights}')['state_dict']

                with torch.no_grad():
                    self.image_map.proj[0].weight.copy_(state_dict['image_proj_weight'])
                    self.image_map.proj[0].bias.copy_(state_dict['image_proj_bias'])
                    self.text_map.proj[0].weight.copy_(state_dict['text_proj_weight'])
                    self.text_map.proj[0].bias.copy_(state_dict['text_proj_bias'])

                if self.freeze_proj:
                    # freeze projection layers
                    for name, p in self.image_map.named_parameters():
                        p.requires_grad_(False)
                    for name, p in self.text_map.named_parameters():
                        p.requires_grad_(False)

            pre_output_input_dim = self.map_dim

        elif self.name == 'text-inv':
            self.text_inv = TextualInversion(self.clip_model, CLIP_IMG_ENC_OUTPUT_DIM_BEFORE_PROJ, self.phi_inv_proj,
                                             self.text_inv_proj, self.post_inv_proj, args.drop_probs, self.phi_freeze,
                                             self.enh_text, self.map_dim, self.num_mapping_layers)

            pre_output_input_dim = self.text_inv.output_dim

        elif self.name == 'text-inv-fusion':
            self.text_inv = TextualInversion(self.clip_model, CLIP_IMG_ENC_OUTPUT_DIM_BEFORE_PROJ, self.phi_inv_proj,
                                             self.text_inv_proj, self.post_inv_proj, args.drop_probs, self.phi_freeze,
                                             self.enh_text, self.map_dim, self.num_mapping_layers)

            self.image_map = LinearProjection(CLIP_IMG_ENC_OUTPUT_DIM_BEFORE_PROJ, self.map_dim,
                                              self.num_mapping_layers, args.drop_probs)

            pre_output_input_dim = self.text_inv.output_dim

        elif self.name == 'text-inv-comb':
            self.comb = Combiner(self.convex_tensor, self.map_dim, self.comb_proj, self.comb_fusion)

            self.text_inv = TextualInversion(self.clip_model, CLIP_IMG_ENC_OUTPUT_DIM_BEFORE_PROJ, self.phi_inv_proj,
                                             self.text_inv_proj, self.post_inv_proj, args.drop_probs, self.phi_freeze,
                                             self.enh_text, self.map_dim, self.num_mapping_layers)

            self.image_map = LinearProjection(CLIP_IMG_ENC_OUTPUT_DIM_BEFORE_PROJ, self.map_dim,
                                              self.num_mapping_layers, args.drop_probs)

            if self.fusion == 'align':
                pre_output_input_dim = self.map_dim
            elif self.fusion == 'concat':
                pre_output_input_dim = 2 * self.map_dim
            else:
                raise ValueError()

            if self.pretrained_proj:
                assert self.num_mapping_layers == 1

                if self.dataset == 'hmc':
                    assert self.map_dim in [1024, 768]
                    weights = f'hmc_{self.map_dim}_projection_embeddings.pt'
                    weights_768 = f'hmc_768_projection_embeddings.pt'

                elif self.dataset == 'harmeme':
                    assert self.map_dim == 768
                    weights_768 = f'harmeme_{self.map_dim}_projection_embeddings.pt'
                    weights = weights_768

                else:
                    raise ValueError()

                state_dict = torch.load(f'{self.pretrained_weights_path}/{weights}')['state_dict']
                state_dict_768 = torch.load(f'{self.pretrained_weights_path}/{weights_768}')['state_dict']

                with torch.no_grad():
                    self.image_map.proj[0].weight.copy_(state_dict['image_proj_weight'])
                    self.image_map.proj[0].bias.copy_(state_dict['image_proj_bias'])
                    self.text_inv.pre_inversion_map[0].weight.copy_(state_dict_768['image_proj_weight'])
                    self.text_inv.pre_inversion_map[0].bias.copy_(state_dict_768['image_proj_bias'])

                if self.freeze_proj:
                    # freeze projection layers
                    for name, p in self.image_map.proj.named_parameters():
                        p.requires_grad_(False)
                    for name, p in self.text_inv.pre_inversion_map.named_parameters():
                        p.requires_grad_(False)
        else:
            raise ValueError()

        pre_output_layers = [nn.Dropout(p=args.drop_probs[1])]
        output_input_dim = pre_output_input_dim

        if self.num_pre_output_layers >= 1:
            pre_output_layers.extend(
                [nn.Linear(pre_output_input_dim, self.map_dim), nn.ReLU(), nn.Dropout(p=args.drop_probs[2])])
            output_input_dim = self.map_dim

        for _ in range(1, self.num_pre_output_layers):
            pre_output_layers.extend(
                [nn.Linear(self.map_dim, self.map_dim), nn.ReLU(), nn.Dropout(p=args.drop_probs[2])])

        self.pre_output = nn.Sequential(*pre_output_layers)
        self.output = nn.Linear(output_input_dim, 1)

        self.cross_entropy_loss = torch.nn.BCEWithLogitsLoss(reduction='mean')

    def forward(self, batch):
        pass

    def compute_CLIP_features_without_proj(self, clip_model, img_input, text_input):

        image_features = clip_model.visual(img_input.type(clip_model.dtype))

        x = clip_model.token_embedding(text_input).type(clip_model.dtype)
        x = x + clip_model.positional_embedding.type(clip_model.dtype)
        x = x.permute(1, 0, 2)
        x = clip_model.transformer(x)
        x = x.permute(1, 0, 2)
        x = clip_model.ln_final(x).type(clip_model.dtype)
        text_features = x[torch.arange(x.shape[0]), text_input.argmax(dim=-1)]

        return image_features, text_features

    def common_step(self, batch):
        if self.fast_process:
            image_features = batch['images']
            text_features = batch['texts']
        else:
            image_features, text_features = self.compute_CLIP_features_without_proj(self.clip_model,
                                                                                    batch['pixel_values'],
                                                                                    batch['texts'])
        if self.enh_text:
            prompt = batch['enhanced_texts']
        else:
            prompt = batch['simple_prompt']

        output = {}

        if self.name in ['hate-clipper', 'adaptation']:
            image_features = self.image_map(image_features)

            text_features = self.text_map(text_features)
            if self.fusion == 'align':
                features = torch.mul(image_features, text_features)
            elif self.fusion == 'concat':
                features = torch.cat([image_features, text_features], dim=1)
            else:
                raise ValueError()

        elif self.name == 'text-only':
            if self.proj_map:
                features = self.text_map(text_features)
            else:
                features = text_features

        elif self.name == 'image-only':
            if self.proj_map:
                features = self.image_map(image_features)
            else:
                features = image_features

        elif self.name == 'sum':
            img_features = self.image_map(image_features)
            txt_features = self.text_map(text_features)
            features = img_features + txt_features

        elif self.name == 'combiner':
            proj_img_features = self.image_map(image_features)
            proj_txt_features = self.text_map(text_features)

            features = self.comb(proj_img_features, proj_txt_features)

        elif self.name == 'text-inv':
            features = self.text_inv(prompt, image_features)

        elif self.name == 'text-inv-fusion':
            features = self.text_inv(prompt, image_features)

            img_projection = self.image_map(image_features)

            if self.fusion == 'concat':
                features = torch.cat([features, img_projection], dim=1)
            elif self.fusion == 'align':
                features = torch.mul(features, img_projection)
            else:
                raise ValueError()

        elif self.name == 'text-inv-comb':
            txt_features = self.text_inv(prompt, image_features)
            print(txt_features.shape)

            img_projection = self.image_map(image_features)
            print(img_projection)

            features = self.comb(img_projection, txt_features)

        else:
            raise ValueError()

        features_pre_output = self.pre_output(features)
        logits = self.output(features_pre_output).squeeze(dim=1)
        print(logits)
        preds_proxy = torch.sigmoid(logits)
        print('preds_proxy:', preds_proxy)
        preds = (preds_proxy >= 0.5).long()
        print("preds:", preds.item())
        output['loss'] = self.cross_entropy_loss(logits, batch['labels'].float())
        output['accuracy'] = self.acc(preds, batch['labels'])
        output['auroc'] = self.auroc(preds_proxy, batch['labels'])

        return output

        prefix = prefix_map[dataloader_idx]

        output = self.common_step(batch)

        self.log(f'{prefix}/accuracy', output['accuracy'])
        self.log(f'{prefix}/auroc', output['auroc'])

        return output

    def training_epoch_end(self, outputs):
        self.acc.reset()
        self.auroc.reset()

    def validation_epoch_end(self, outputs):
        self.acc.reset()
        self.auroc.reset()

    def test_epoch_end(self, outputs):
        self.acc.reset()
        self.auroc.reset()

    def configure_optimizers(self):
        param_dicts = [
            {"params": [p for n, p in self.named_parameters() if p.requires_grad]}
        ]
        optimizer = torch.optim.AdamW(param_dicts, lr=self.lr, weight_decay=self.weight_decay)

        return optimizer


def create_model(args):
    model = HateClassifier(args=args)
    return model

In [ ]:
args = argparse.Namespace(
    dataset='hmc',
    image_size=224,
    num_mapping_layers=1,
    map_dim=1024,
    fusion='align',
    num_pre_output_layers=1,
    drop_probs=[0.2, 0.4, 0.1],
    gpus='0',
    limit_train_batches=1.0,
    limit_val_batches=1.0,
    max_steps=-1,
    max_epochs=-1,
    log_every_n_steps=25,
    val_check_interval=1.0,
    batch_size=1,
    lr=1e-4,
    weight_decay=1e-4,
    gradient_clip_val=0.1,
    proj_map=False,
    pretrained_proj_weights=True,
    freeze_proj_layers=True,
    comb_proj=True,
    comb_fusion='align',
    convex_tensor=False,
    text_inv_proj=True,
    phi_inv_proj=True,
    post_inv_proj=True,
    enh_text=True,
    phi_freeze=True,
    name='text-inv-comb',
    pretrained_model='hmc_text-inv-comb_best.ckpt',
    reproduce=True,
    print_model=True,
    fast_process=False
)

model = HateClassifier.load_from_checkpoint(f'resources/pretrained_models/{args.pretrained_model}', args=args,map_location='cuda:0')
model.to('cuda')
model.eval()

transform = transforms.Compose([
    transforms.Resize((args.image_size, args.image_size)),
    transforms.CenterCrop(args.image_size),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.48145466, 0.4578275, 0.40821073], std=[0.26862954, 0.26130258, 0.27577711])
])

In [ ]:
# %%time
image_path = '/content/01235.png'

image = Image.open(image_path).convert('RGB')
image_tensor = transform(image).unsqueeze(0)
image_tensor = image_tensor.to('cuda')

# ImpaintingResult = transform_image('/content/memetest.jpg')
# CaptioningResult = process_meme_caption(image_path)
OcrResult = MyOCR(image_path)
# CaptioningResult = CaptioningFunc(image_path)

batch = {
    'pixel_values': image_tensor,
    'texts': clip.tokenize('a photo of $ ,', context_length=77).to('cuda'),
    'labels': torch.tensor([1]).to('cuda'),
    'enhanced_texts': clip.tokenize(f'a picture of $ , {OcrResult}', context_length=77).to('cuda')
}

with torch.no_grad():
    output = model.common_step(batch)

#Demo

In [ ]:
input_data = "a better question what do american voters need to do to defeat antifa amp their democrat sponsors" # @param {type:"string"}
import magic
mime = magic.Magic(mime=True)

# Check if the variable is a non-empty string
if isinstance(input_data, str):
    # Check if the string represents an existing file
    if os.path.exists(input_data):
        dataType = mime.from_file(input_data)
        # print(dataType)
        if dataType.startswith("image"):
            pred = reader.readtext(input_data, detail = 0)
            pred = ' '.join(pred)
            # output = perform_inference(input_data)
            if pred == "":
                print("Data type: pure image")
                output = perform_inference(input_data)
            else:
                print("Data type: meme")
                start = time.time()
                caption = process_meme_caption(input_data)
                print("meme content:", pred + '. ' + caption)
                output = perform_inference(pred + '. ' + caption, meme=True)
        elif dataType.startswith("text"):
            print("Data type: text")
            with open(input_data, 'r') as text_file:
                text_data = text_file.read()
            output = perform_inference(text_data)
        elif dataType.startswith("audio"):
            print("Data type: audio")
            text = audio_model.transcribe(input_data)
            print("audio content:", text['text'])
            output = perform_inference(text['text'])
    else:
        print("Data type: text")
        output = perform_inference(input_data)

Data type: text
Prediction: NonBully
